# Firefox driver

In [ ]:
# %run "../../../../biofouling_models/Web scraping and data/SABIO/complete_sabio.py"
%run ../bigg_sabio/scrapper.py
scraping = SABIO_scraping()
scraping.main('../bigg_models/Ecoli core, BiGG, indented.json', 'test_Ecoli-2')

Estimated completion of scraping data for test_Ecoli-2: 2022-01-28 08:04:43.644229, in 4.75 hours
The Phosphofructokinase is duplicated in the model
The Pyruvate formate lyase is duplicated in the model
The Glucose-6-phosphate isomerase is duplicated in the model
The Phosphoglycerate kinase is duplicated in the model
The 6-phosphogluconolactonase is duplicated in the model
The Acetaldehyde dehydrogenase (acetylating) is duplicated in the model
The 2 oxoglutarate reversible transport via symport is duplicated in the model
The Phosphoglycerate mutase is duplicated in the model
The Phosphate reversible transport via symport is duplicated in the model


In [1]:
from pprint import pprint
pprint(locals())
values = {
    'a': 120,
    'b': 10
}
locals().update(values)
pprint(locals())
print(eval('a*b'))

{'In': ['',
        'from pprint import pprint\n'
        'pprint(locals())\n'
        'values = {\n'
        "    'a': 120,\n"
        "    'b': 10\n"
        '}\n'
        'locals().update(values)\n'
        'pprint(locals())\n'
        "print(eval('a*b'))"],
 'Out': {},
 '_': '',
 '__': '',
 '___': '',
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__loader__': None,
 '__name__': '__main__',
 '__package__': None,
 '__spec__': None,
 '_dh': ['C:\\Users\\Andrew Freiburger\\Dropbox\\My PC '
         '(DESKTOP-M302P50)\\Documents\\UVic Civil '
         'Engineering\\dFBA\\BiGG_SABIO\\examples'],
 '_i': '',
 '_i1': 'from pprint import pprint\n'
        'pprint(locals())\n'
        'values = {\n'
        "    'a': 120,\n"
        "    'b': 10\n"
        '}\n'
        'locals().update(values)\n'
        'pprint(locals())\n'
        "print(eval('a*b'))",
 '_ih'

In [1]:
for x: str in range(10):
    print(x)
print(type(x))

SyntaxError: invalid syntax (<ipython-input-1-f743a63a004c>, line 1)

In [1]:
for x in range(10):
    print(x, end='\r')

In [ ]:
products = []
products.append(6)
products[-1] = 4
print(products)

In [5]:
import pandas

df = pandas.DataFrame(data = [[4,5], [7,8]])

display(df)

print(df[0][0])

,0,1
0,4,5
1,7,8


4


In [13]:
# -*- coding: utf-8 -*-
"""
@authors: Ethan Chan, Matthew Freiburger
"""
# Import libraries
import pandas as pd
from glob import glob
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
from itertools import islice
import re



class SABIO_scraping():
#     __slots__ = (str(x) for x in [progress_file_prefix, xls_download_prefix, scraped_xls_prefix, scraped_entryids_prefix, sel_xls_download_path, processed_xls, entry_json, scraped_model, bigg_model_name_suffix, sub_directory_path, progress_file_path, xls_download_path, scraped_xls_file_path, scraped_entryids_file_path, xls_csv_file_path, entryids_json_file_path, scraped_model_json_file_path, bigg_model, step_number, cwd])
    
    def __init__(self,source='sabio'):

        self.source = 'sabio'
            
        self.count = 0
            
        self.parameters = {}
        self.parameters['general_delay'] = 2
        self.variables = {}
        self.paths = {}

    #Clicks a HTML element with selenium by id
    def click_element_id(self,n_id):
        element = self.driver.find_element_by_id(n_id)
        element.click()
        time.sleep(self.parameters['general_delay'])
        
    def wait_for_id(self,n_id):
        while True:
            try:
                element = self.driver.find_element_by_id(n_id)
                break
            except:
                time.sleep(self.parameters['general_delay'])
        

    #Selects a choice from a HTML dropdown element with selenium by id
    def select_dropdown_id(self,n_id, n_choice):
        element = Select(self.driver.find_element_by_id(n_id))
        element.select_by_visible_text(n_choice)
        time.sleep(self.parameters['general_delay'])

    def fatal_error_handler(self,message):
        print("Error: " + message)
        print("Exiting now...")
        exit(0)

    """
    --------------------------------------------------------------------
        STEP 0: GET BIGG MODEL TO SCRAPE AND SETUP DIRECTORIES AND PROGRESS FILE
    --------------------------------------------------------------------    
    """

    def start(self,): # find the BiGG model that will be scraped
        while True:
            bigg_model_path = input("Specify the BIGG Model JSON file path: ")

            if os.path.exists(bigg_model_path) and bigg_model_path[-5:] == ".json":
                try:
                    self.model = json.load(open(bigg_model_path))
                    bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()
                    break
                except:
                    pass

#         self.model = json.load(open('BiGG model of S. aureus.json'))
#         bigg_model_path = 'BiGG model of S. aureus.json'
#         bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()

        # define the paths
        self.paths['cwd'] = os.path.dirname(os.path.realpath(bigg_model_path))
        print(self.paths['cwd'])
        self.paths['sub_directory_path'] = os.path.join(self.paths['cwd'],f"scraping-{bigg_model_name}")
        if not os.path.isdir(self.paths['sub_directory_path']):        
            os.mkdir(self.paths['sub_directory_path'])
                    
        self.variables['scraped_xls'] = {}
        self.variables['scraped_entryids'] = {}
        self.paths['scraped_model_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_model") + ".json"
        self.paths['sel_xls_download_path'] = os.path.join(self.paths['sub_directory_path'],"downloaded_xls")
        
        
        
        self.step_number = 1
        
        self.paths['progress_file_path'] = os.path.join(self.paths['sub_directory_path'], "current_progress") + '.txt'
        if os.path.exists(self.paths['progress_file_path']):
            f = open(self.paths['progress_file_path'], "r")
            self.step_number = int(f.read(1))
            if not re.search('[1-5]',str(self.step_number)):
                self.fatal_error_handler("Progress file malformed. Please delete and restart")
        else:
            self.step_number = 1
            self.progress_update(self.step_number)

        self.paths['xls_download_path'] = os.path.join(self.paths['sub_directory_path'], 'downloaded_xls') 
        if not os.path.isdir(self.paths['xls_download_path']):
            os.mkdir(self.paths['xls_download_path'])

        self.paths['scraped_xls_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_xls") + ".json"
        if os.path.exists(self.paths['scraped_xls_file_path']):
            f = open(self.paths['scraped_xls_file_path'], "r")
            self.variables['scraped_xls'] = json.load(f)
            f.close()

        self.paths['scraped_entryids_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_entryids") + ".json"
        if os.path.exists(self.paths['scraped_entryids_file_path']):
            f = open(self.paths['scraped_entryids_file_path'], "r")
            self.variables['scraped_entryids'] = json.load(f)
            f.close()

        self.paths['entryids_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "entryids_json") + ".json"
        if os.path.exists(self.paths['entryids_json_file_path']):
            f = open(self.paths['entryids_json_file_path'], "r")
            self.variables['entry_id_json_out'] = json.load(f)
            f.close()

    """
    --------------------------------------------------------------------
        STEP 1: SCRAPE SABIO WEBSITE BY DOWNLOAD XLS FOR GIVEN REACTIONS IN BIGG MODEL
    --------------------------------------------------------------------    
    """

    def scrape_xls(self,reaction_identifier, search_option):
        
        
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")
        
        self.wait_for_id("resetbtn")
        
        self.click_element_id("resetbtn")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", search_option)
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(reaction_identifier)  
        time.sleep(self.parameters['general_delay'])  
        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        result_num = ""
        try: 
            result_num_ele = self.driver.find_element_by_id("numberofKinLaw")
            for char in result_num_ele.text:
                if re.search('[0-9]', char):
                    result_num = result_num + char

            result_num = int(result_num)
        except:
            #self.driver.close()
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        time.sleep(self.parameters['general_delay'])

        self.select_dropdown_id("max", "100")
        element = Select(self.driver.find_element_by_id("max"))
        element.select_by_visible_text("100")

        time.sleep(self.parameters['general_delay'])

        if result_num > 0 and result_num <= 100:
            self.click_element_id("allCheckbox")
            time.sleep(self.parameters['general_delay'])
        elif result_num > 100:
            self.click_element_id("allCheckbox")
            for i in range(int(result_num/100)):
                element = self.driver.find_element_by_xpath("//*[@class = 'nextLink']")
                element.click()
                time.sleep(self.parameters['general_delay'])
                self.click_element_id("allCheckbox")
                time.sleep(self.parameters['general_delay'])
        else:
            #self.driver.close()
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        self.driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")

        time.sleep(self.parameters['general_delay']*7.5)

        self.click_element_id("excelExport")

        time.sleep(self.parameters['general_delay']*2.5)

        #self.driver.close()

        return True
    
    def expand_shadow_element(self, element):
        shadow_root = self.driver.execute_script('return arguments[0].shadowRoot', element)
        return shadow_root


    def scrape_bigg_xls(self,):
        """
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path']}
        chrome_options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)
        
        self.driver.get("chrome://settings/security")
        

        
        root = self.driver.find_element_by_tag_name("settings-ui")
        shadow_root = self.expand_shadow_element(root)
        
        root1 = shadow_root.find_element_by_tag_name("settings-main")
        shadow_root1 = self.expand_shadow_element(root1)
        
        root2 = shadow_root1.find_element_by_tag_name("settings-basic-page")
        shadow_root2 = self.expand_shadow_element(root2)
        
        root3 = shadow_root2.find_element_by_tag_name("settings-privacy-page")
        shadow_root3 = self.expand_shadow_element(root3)
        
        root4 = shadow_root3.find_element_by_tag_name("settings-security-page")
        shadow_root4 = self.expand_shadow_element(root4)
        
        root5 = shadow_root4.find_element_by_css_selector("#safeBrowsingStandard")
        shadow_root5 = self.expand_shadow_element(root5)
        
        security_button = shadow_root5.find_element_by_css_selector("div.disc-border")
        
        security_button.click()
        """
        
        """
        fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
        fp.set_preference("browser.download.folderList",2)
        fp.set_preference("browser.download.dir", self.paths['cwd'] + self.paths['sel_xls_download_path'])
        """
        #print(self.paths['cwd'] + self.paths['sel_xls_download_path'])
        fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
        fp.set_preference("browser.download.folderList", 2)
        fp.set_preference("browser.download.manager.showWhenStarting", False)
        #temp_path = f'⁦C:\\Users\\Ethan\\Documents\\Python\\Polar Bar Chart\\sabioawesome\\scraping-test_file\\downloaded_xls⁩'
        #fp.set_preference("browser.download.dir", self.paths["cwd"] + self.paths["sel_xls_download_path"])
        print( self.paths["sel_xls_download_path"])
        fp.set_preference("browser.download.dir", self.paths["sel_xls_download_path"])
        fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
        self.driver = webdriver.Firefox(firefox_profile=fp, executable_path="geckodriver.exe") 
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        
        for reaction in self.model["reactions"]:
            if not reaction["name"] in self.variables['scraped_xls']:
                ids_to_try = reaction["annotation"]

                success_flag = False
                annotation_search_pairs = {"sabiork":"SabioReactionID", "metanetx.reaction":"MetaNetXReactionID", "ec-code":"ECNumber", "kegg.reaction":"KeggReactionID", "rhea":"RheaReactionID"}
                for annotation in annotation_search_pairs:
                    if not success_flag:
                        if annotation in ids_to_try:


                            for id_to_try in ids_to_try[annotation]:
#                                 success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                try:
                                    success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                except:
                                    success_flag = False
                    else:
                        break

                if not success_flag:
                    try:
                        success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    except:
                        success_flag = False

                json_dict_key = reaction["name"].replace("\"", "")
                if success_flag:
                    self.variables['scraped_xls'][json_dict_key] = "yes"
                else:
                    self.variables['scraped_xls'][json_dict_key] = "no"
                    
                self.count += 1
                print("Reactions searched: " + str(self.count) + "/" + str(len(self.model["reactions"])), end='\r')
                    
                    

            with open(self.paths['scraped_xls_file_path'], 'w') as outfile:
                json.dump(self.variables['scraped_xls'], outfile, indent = 4)   
                outfile.close()
                
        self.step_number = 2
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 2: GLOB EXPORTED XLS FILES TOGETHER
    --------------------------------------------------------------------
    """

    def glob_xls_files(self,):
#         scraped_sans_parentheses_enzymes = glob('./{}/*.xls'.format(self.paths['xls_download_path']))
        scraped_sans_parentheses_enzymes = glob(os.path.join(self.paths['xls_download_path'], '*.xls'))
        total_dataframes = []
        for file in scraped_sans_parentheses_enzymes:
            #file_name = os.path.splitext(os.path.basename(file))[0]
            dfn = pd.read_excel(file)
            total_dataframes.append(dfn)

        # combine the total set of dataframes
        combined_df = pd.DataFrame()
        combined_df = pd.concat(total_dataframes)
        combined_df = combined_df.fillna(' ')
        combined_df = combined_df.drop_duplicates()

        # export the dataframe
        csv_path = os.path.join(self.paths['sub_directory_path'], "proccessed-xls") + ".csv"
        combined_df.to_csv(csv_path)
        
        # update the 
        self.step_number = 3
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 3: SCRAPE ADDITIONAL DATA BY ENTRYID
    --------------------------------------------------------------------    
    """

    def scrape_entry_id(self,entry_id):
        global entry_id_json_out

        entry_id = str(entry_id)

        fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
        fp.set_preference("browser.download.folderList",2)
        fp.set_preference("browser.download.dir", self.paths['cwd'] + self.paths['sel_xls_download_path'])
        self.driver = webdriver.Firefox(firefox_profile=fp, executable_path="geckodriver.exe") 
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", "EntryID")
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(entry_id)

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id(entry_id + "img")

        time.sleep(self.parameters['general_delay'])

        self.driver.switch_to.frame(self.driver.find_element_by_xpath("//iframe[@name='iframe_" + entry_id + "']"))
        element = self.driver.find_element_by_xpath("//table")
        html_source = element.get_attribute('innerHTML')
        
        table_df = pd.read_html(html_source)
        reaction_parameters_df = pd.DataFrame()
        counter = 0
        parameters_json = {}
        for df in table_df:
            try:
                if df[0][0] == "Parameter":
                    reaction_parameters_df = table_df[counter]
            except:
                self.driver.close()
                return parameters_json
            counter += 1
            
        parameter_name = ""
        for i in range(len(reaction_parameters_df[0])-2):
            parameter_name = reaction_parameters_df[0][i+2]
            inner_parameters_json = {}
            for j in range(len(reaction_parameters_df)-3):
                inner_parameters_json[reaction_parameters_df[j+1][1]] = reaction_parameters_df[j+1][i+2]

            parameters_json[parameter_name] = inner_parameters_json

        self.driver.close()

        return parameters_json


    def scrape_entryids(self,):
        global entry_id_json_out

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])
        entryids = sabio_xls_df["EntryID"].unique().tolist()

        for entryid in entryids:
            if not entryid in self.variables['scraped_entryids']:
                try:
                    entry_id_json_out[str(entryid)] = self.scrape_entry_id(entryid)
                    self.variables['scraped_entryids'][entryid] = "yes"
                except:
                    self.variables['scraped_entryids'][entryid] = "no"
            with open(self.paths["(scraped_entryids_file_path"], 'w') as outfile:
                json.dump(self.variables['scraped_entryids'], outfile, indent = 4)   
                outfile.close()
            with open(self.paths["entryids_json_file_path"], 'w') as f:
                json.dump(entry_id_json_out, f, indent = 4)        
                f.close()
        
        self.step_number = 4
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 4: COMBINE ENZYME AND ENTRYID DATA INTO JSON FILE
    --------------------------------------------------------------------
    """

    def combine_data(self,):

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])

        # Opening JSON file
        with open(self.paths['entryids_json_file_path']) as json_file:
            entry_id_data = json.load(json_file)

        enzymenames = sabio_xls_df["Enzymename"].unique().tolist()
        enzyme_dict = {}
        missing_entry_ids = []
        parameters = {}

        for enzyme in enzymenames:
            sabio_grouped_enzyme_df = sabio_xls_df.loc[sabio_xls_df["Enzymename"] == enzyme]
            dict_to_append = {}
            reactions = sabio_grouped_enzyme_df["Reaction"].unique().tolist()
            for reaction in reactions:
                dict_reactions_to_append = {}
                sabio_grouped_reactions_df = sabio_grouped_enzyme_df.loc[sabio_grouped_enzyme_df["Reaction"] == reaction]
                entryids = sabio_grouped_reactions_df["EntryID"].unique().tolist()

                for entryid in entryids:
                    entry_ids_df = sabio_grouped_reactions_df.loc[sabio_grouped_reactions_df["EntryID"] == entryid]
                    dict_entryid_to_append = {}
                    head_of_df = entry_ids_df.head(1).squeeze()
                    entry_id_flag = True
                    parameter_info = {}

                    try:
                        parameter_info = entry_id_data[str(entryid)]
                        dict_entryid_to_append["Parameters"] = parameter_info
                    except:
                        missing_entry_ids.append(str(entryid))
                        entry_id_flag = False
                        dict_entryid_to_append["Parameters"] = "NaN"

                    rate_law = head_of_df["Rate Equation"]
                    bad_rate_laws = ["unknown", "", "-"]

                    if not rate_law in bad_rate_laws:                    
                        dict_entryid_to_append["RateLaw"] = rate_law
                        dict_entryid_to_append["SubstitutedRateLaw"] = rate_law
                    else:
                        dict_entryid_to_append["RateLaw"] = "NaN"
                        dict_entryid_to_append["SubstitutedRateLaw"] = "NaN"

                    if entry_id_flag:

                        fields_to_copy = ["Buffer", "Product", "PubMedID", "Publication", "pH", "Temperature", "Enzyme Variant", "UniProtKB_AC", "Organism", "KineticMechanismType", "SabioReactionID"]
                        for field in fields_to_copy:  
                            dict_entryid_to_append[field] = head_of_df[field]
                        dict_reactions_to_append[entryid] = dict_entryid_to_append
                        dict_entryid_to_append["Substrates"] = head_of_df["Substrate"].split(";")
                        out_rate_law = rate_law
                        if not rate_law in bad_rate_laws:                    
                            substrates = head_of_df["Substrate"].split(";")

                            stripped_string = re.sub('[0-9]', '', rate_law)

                            variables = re.split("\^|\*|\+|\-|\/|\(|\)| ", stripped_string)
                            variables = ' '.join(variables).split()

                            start_value_permutations = ["start value", "start val."]
                            substrates_key = {}
                            for var in variables:
                                if var in parameter_info:
                                    for permutation in start_value_permutations:
                                        try:
                                            if var == "A" or var == "B":
                                                substrates_key[var] = parameter_info[var]["species"]
                                            else:
                                                value = parameter_info[var][permutation]
                                                if value != "-" and value != "" and value != " ":
                                                    out_rate_law = out_rate_law.replace(var, parameter_info[var][permutation])
                                        except:
                                            pass

                            dict_entryid_to_append["RateLawSubstrates"] = substrates_key
                            dict_entryid_to_append["SubstitutedRateLaw"] = out_rate_law

                dict_to_append[reaction] = dict_reactions_to_append

            enzyme_dict[enzyme] = dict_to_append

        with open(self.paths["scraped_model_json_file_path"], 'w') as f:
            json.dump(enzyme_dict, f, indent=4)
        
        self.step_number = 5
        self.progress_update(self.step_number)
        
    def progress_update(self, step):
        if not re.search('[0-5]', str(step)):
            print(f'--> ERROR: The {step} step is not acceptable.')
        f = open(self.paths['progress_file_path'], "w")
        f.write(str(step))
        f.close()

    def main(self,):
        self.start()

        while True:
            if self.step_number == 1:
                self.scrape_bigg_xls()
            elif self.step_number == 2:
                self.glob_xls_files()
            elif self.step_number == 3:
                self.scrape_entryids()
            elif self.step_number == 4:
                self.combine_data()
            elif self.step_number == 5:
                print("Execution complete. Scraper finished.")
                break
            


In [ ]:
# ./BiGG_models/BiGG model of S. aureus.json
# -*- coding: utf-8 -*-
"""
@authors: Ethan Chan, Matthew Freiburger
"""
# Import libraries
import pandas as pd
import glob
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
from itertools import islice
import re



class SABIO_scraping():
#     __slots__ = (str(x) for x in [progress_file_prefix, xls_download_prefix, scraped_xls_prefix, scraped_entryids_prefix, sel_xls_download_path, processed_xls, entry_json, scraped_model, bigg_model_name_suffix, sub_directory_path, progress_file_path, xls_download_path, scraped_xls_file_path, scraped_entryids_file_path, xls_csv_file_path, entryids_json_file_path, scraped_model_json_file_path, bigg_model, step_number, cwd])
    
    def __init__(self):           
        self.count = 0
            
        self.parameters = {}
        self.parameters['general_delay'] = 2
        self.variables = {}
        self.paths = {}

    #Clicks a HTML element with selenium by id
    def click_element_id(self,n_id):
        element = self.driver.find_element_by_id(n_id)
        element.click()
        time.sleep(self.parameters['general_delay'])

    #Selects a choice from a HTML dropdown element with selenium by id
    def select_dropdown_id(self,n_id, n_choice):
        element = Select(self.driver.find_element_by_id(n_id))
        element.select_by_visible_text(n_choice)
        time.sleep(self.parameters['general_delay'])

    def fatal_error_handler(self,message):
        print("Error: " + message)
        print("Exiting now...")
        exit(0)

    """
    --------------------------------------------------------------------
        STEP 0: GET BIGG MODEL TO SCRAPE AND SETUP DIRECTORIES AND PROGRESS FILE
    --------------------------------------------------------------------    
    """

    def start(self,source='sabio',): # find the BiGG model that will be scraped
        while True:
            bigg_model_path = input("Specify the BIGG Model JSON file path: ")

            if os.path.exists(bigg_model_path) and bigg_model_path[-5:] == ".json":
                try:
                    self.model = json.load(open(bigg_model_path))
                    bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()
                    break
                except:
                    pass

#         self.model = json.load(open('BiGG model of S. aureus.json'))
#         bigg_model_path = 'BiGG model of S. aureus.json'
#         bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()

        # define the paths
        self.paths['cwd'] = os.path.dirname(os.path.realpath(bigg_model_path))
        print(self.paths['cwd'])
        self.paths['sub_directory_path'] = os.path.join(self.paths['cwd'],f"scraping-{bigg_model_name}")
        if not os.path.isdir(self.paths['sub_directory_path']):        
            os.mkdir(self.paths['sub_directory_path'])
                    
        self.variables['scraped_xls'] = {}
        self.variables['scraped_entryids'] = {}
        self.paths['scraped_model_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_model") + ".json"
        self.paths['sel_xls_download_path'] = os.path.join(self.paths['sub_directory_path'],"xls_download")
        
        self.step_number = 1
        
        self.paths['progress_file_path'] = os.path.join(self.paths['sub_directory_path'], "current_progress") + '.txt'
        if os.path.exists(self.paths['progress_file_path']):
            f = open(self.paths['progress_file_path'], "r")
            self.step_number = int(f.read(1))
            if not re.search('[1-5]',str(self.step_number)):
                self.fatal_error_handler("Progress file malformed. Please delete and restart")
        else:
            self.step_number = 1
            self.progress_update(self.step_number)

        self.paths['xls_download_path'] = os.path.join(self.paths['sub_directory_path'], 'downloaded_xls') 
        if not os.path.isdir(self.paths['xls_download_path']):
            os.mkdir(self.paths['xls_download_path'])

        self.paths['scraped_xls_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_xls") + ".json"
        if os.path.exists(self.paths['scraped_xls_file_path']):
            f = open(self.paths['scraped_xls_file_path'], "r")
            self.variables['scraped_xls'] = json.load(f)
            f.close()

        self.paths['scraped_entryids_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_entryids") + ".json"
        if os.path.exists(self.paths['scraped_entryids_file_path']):
            f = open(self.paths['scraped_entryids_file_path'], "r")
            self.variables['scraped_entryids'] = json.load(f)
            f.close()

        self.paths['entryids_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "entryids_json") + ".json"
        if os.path.exists(self.paths['entryids_json_file_path']):
            f = open(self.paths['entryids_json_file_path'], "r")
            self.variables['entry_id_json_out'] = json.load(f)
            f.close()
            
        fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
        fp.set_preference("browser.download.folderList",2)
        self.driver = webdriver.Firefox(firefox_profile=fp, executable_path="geckodriver.exe") 
        
        if source == 'sabio':
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            self.source = 'sabio'

    """
    --------------------------------------------------------------------
        STEP 1: SCRAPE SABIO WEBSITE BY DOWNLOAD XLS FOR GIVEN REACTIONS IN BIGG MODEL
    --------------------------------------------------------------------    
    """

    def scrape_xls(self,reaction_identifier, search_option):
        self.count += 1
        print(self.count, end='\r')
        
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")
        self.click_element_id("resetbtn")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", search_option)
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(reaction_identifier)  
        time.sleep(self.parameters['general_delay'])  
        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        result_num = ""
        try: 
            result_num_ele = self.driver.find_element_by_id("numberofKinLaw")
            for char in result_num_ele.text:
                if re.search('[0-9]', char):
                    result_num = result_num + char

            result_num = int(result_num)
        except:
            #self.driver.close()
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        time.sleep(self.parameters['general_delay'])

        self.select_dropdown_id("max", "100")
        element = Select(self.driver.find_element_by_id("max"))
        element.select_by_visible_text("100")

        time.sleep(self.parameters['general_delay'])

        if result_num > 0 and result_num <= 100:
            self.click_element_id("allCheckbox")
            time.sleep(self.parameters['general_delay'])
        elif result_num > 100:
            self.click_element_id("allCheckbox")
            for i in range(int(result_num/100)):
                element = self.driver.find_element_by_xpath("//*[@class = 'nextLink']")
                element.click()
                time.sleep(self.parameters['general_delay'])
                self.click_element_id("allCheckbox")
                time.sleep(self.parameters['general_delay'])
        else:
            #self.driver.close()
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        self.driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")

        time.sleep(self.parameters['general_delay']*7.5)

        self.click_element_id("excelExport")

        time.sleep(self.parameters['general_delay']*2.5)

        #self.driver.close()

        return True
    
    def expand_shadow_element(self, element):
        shadow_root = self.driver.execute_script('return arguments[0].shadowRoot', element)
        return shadow_root


    def scrape_bigg_xls(self,):
        """
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path']}
        chrome_options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)
        
        self.driver.get("chrome://settings/security")
        

        
        root = self.driver.find_element_by_tag_name("settings-ui")
        shadow_root = self.expand_shadow_element(root)
        
        root1 = shadow_root.find_element_by_tag_name("settings-main")
        shadow_root1 = self.expand_shadow_element(root1)
        
        root2 = shadow_root1.find_element_by_tag_name("settings-basic-page")
        shadow_root2 = self.expand_shadow_element(root2)
        
        root3 = shadow_root2.find_element_by_tag_name("settings-privacy-page")
        shadow_root3 = self.expand_shadow_element(root3)
        
        root4 = shadow_root3.find_element_by_tag_name("settings-security-page")
        shadow_root4 = self.expand_shadow_element(root4)
        
        root5 = shadow_root4.find_element_by_css_selector("#safeBrowsingStandard")
        shadow_root5 = self.expand_shadow_element(root5)
        
        security_button = shadow_root5.find_element_by_css_selector("div.disc-border")
        
        security_button.click()
        """
        
        fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
        fp.set_preference("browser.download.folderList",2)
        fp.set_preference("browser.download.dir", self.paths['cwd'] + self.paths['sel_xls_download_path'])
        driver = webdriver.Firefox(firefox_profile=fp, executable_path="geckodriver.exe") 
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        
        for reaction in self.model["reactions"]:
            if not reaction["name"] in self.variables['scraped_xls']:
                ids_to_try = reaction["annotation"]

                success_flag = False
                annotation_search_pairs = {"sabiork":"SabioReactionID", "metanetx.reaction":"MetaNetXReactionID", "ec-code":"ECNumber", "kegg.reaction":"KeggReactionID", "rhea":"RheaReactionID"}
                for annotation in annotation_search_pairs:
                    if not success_flag:
                        if annotation in ids_to_try:


                            for id_to_try in ids_to_try[annotation]:
#                                 success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                try:
                                    success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                except:
                                    success_flag = False
                    else:
                        break

                if not success_flag:
                    try:
                        success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    except:
                        success_flag = False

                json_dict_key = reaction["name"].replace("\"", "")
                if success_flag:
                    self.variables['scraped_xls'][json_dict_key] = "yes"
                else:
                    self.variables['scraped_xls'][json_dict_key] = "no"
                    
                    

            with open(self.paths['scraped_xls_file_path'], 'w') as outfile:
                json.dump(self.variables['scraped_xls'], outfile, indent = 4)   
                outfile.close()
                
        self.step_number = 2
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 2: GLOB EXPORTED XLS FILES TOGETHER
    --------------------------------------------------------------------
    """

    def glob_xls_files(self,):
        scraped_sans_parentheses_enzymes = glob.glob('./{}/*.xls'.format(self.paths['xls_download_path']))
        total_dataframes = []
        for file in scraped_sans_parentheses_enzymes:
            #file_name = os.path.splitext(os.path.basename(file))[0]
            dfn = pd.read_excel(file)
            total_dataframes.append(dfn)

        # combine the total set of dataframes
        combined_df = pd.DataFrame()
        combined_df = pd.concat(total_dataframes)
        combined_df = combined_df.fillna(' ')
        combined_df = combined_df.drop_duplicates()

        # export the dataframe
        csv_path = os.path.join(self.paths['sub_directory_path'], "proccessed-xls") + ".csv"
        combined_df.to_csv(csv_path)
        
        # update the 
        self.step_number = 3
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 3: SCRAPE ADDITIONAL DATA BY ENTRYID
    --------------------------------------------------------------------    
    """

    def scrape_entry_id(self,entry_id):
        global entry_id_json_out

        entry_id = str(entry_id)

        self.driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(general_delay)

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", "EntryID")
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(entry_id)

        time.sleep(general_delay)

        self.click_element_id("addsearch")

        time.sleep(general_delay)

        self.click_element_id(entry_id + "img")

        time.sleep(general_delay)

        self.driver.switch_to.frame(self.driver.find_element_by_xpath("//iframe[@name='iframe_" + entry_id + "']"))
        element = self.driver.find_element_by_xpath("//table")
        html_source = element.get_attribute('innerHTML')
        
        table_df = pd.read_html(html_source)
        reaction_parameters_df = pd.DataFrame()
        counter = 0
        parameters_json = {}
        for df in table_df:
            try:
                if df[0][0] == "Parameter":
                    reaction_parameters_df = table_df[counter]
            except:
                self.driver.close()
                return parameters_json
            counter += 1
            
        parameter_name = ""
        for i in range(len(reaction_parameters_df[0])-2):
            parameter_name = reaction_parameters_df[0][i+2]
            inner_parameters_json = {}
            for j in range(len(reaction_parameters_df)-3):
                inner_parameters_json[reaction_parameters_df[j+1][1]] = reaction_parameters_df[j+1][i+2]

            parameters_json[parameter_name] = inner_parameters_json

        self.driver.close()

        return parameters_json


    def scrape_entryids(self,):
        global entry_id_json_out

        sabio_xls_df = pd.read_csv(xls_csv_file_path)
        entryids = sabio_xls_df["EntryID"].unique().tolist()

        for entryid in entryids:
            if not entryid in scraped_entryids:
                try:
                    entry_id_json_out[str(entryid)] = scrape_entry_id(entryid)
                    self.variables['scraped_entryids'][entryid] = "yes"
                except:
                    self.variables['scraped_entryids'][entryid] = "no"
            with open(scraped_entryids_file_path, 'w') as outfile:
                json.dump(self.variables['scraped_entryids'], outfile, indent = 4)   
                outfile.close()
            with open(entryids_json_file_path, 'w') as f:
                json.dump(entry_id_json_out, f, indent = 4)        
                f.close()
        
        self.step_number = 4
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 4: COMBINE ENZYME AND ENTRYID DATA INTO JSON FILE
    --------------------------------------------------------------------
    """

    def combine_data(self,):

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])

        # Opening JSON file
        with open(self.paths['entryids_json_file_path']) as json_file:
            entry_id_data = json.load(json_file)

        enzymenames = sabio_xls_df["Enzymename"].unique().tolist()
        enzyme_dict = {}
        missing_entry_ids = []
        parameters = {}

        for enzyme in enzymenames:
            sabio_grouped_enzyme_df = sabio_xls_df.loc[sabio_xls_df["Enzymename"] == enzyme]
            dict_to_append = {}
            reactions = sabio_grouped_enzyme_df["Reaction"].unique().tolist()
            for reaction in reactions:
                dict_reactions_to_append = {}
                sabio_grouped_reactions_df = sabio_grouped_enzyme_df.loc[sabio_grouped_enzyme_df["Reaction"] == reaction]
                entryids = sabio_grouped_reactions_df["EntryID"].unique().tolist()

                for entryid in entryids:
                    entry_ids_df = sabio_grouped_reactions_df.loc[sabio_grouped_reactions_df["EntryID"] == entryid]
                    dict_entryid_to_append = {}
                    head_of_df = entry_ids_df.head(1).squeeze()
                    entry_id_flag = True
                    parameter_info = {}

                    try:
                        parameter_info = entry_id_data[str(entryid)]
                        dict_entryid_to_append["Parameters"] = parameter_info
                    except:
                        missing_entry_ids.append(str(entryid))
                        entry_id_flag = False
                        dict_entryid_to_append["Parameters"] = "NaN"

                    rate_law = head_of_df["Rate Equation"]
                    bad_rate_laws = ["unknown", "", "-"]

                    if not rate_law in bad_rate_laws:                    
                        dict_entryid_to_append["RateLaw"] = rate_law
                        dict_entryid_to_append["SubstitutedRateLaw"] = rate_law
                    else:
                        dict_entryid_to_append["RateLaw"] = "NaN"
                        dict_entryid_to_append["SubstitutedRateLaw"] = "NaN"

                    if entry_id_flag:

                        fields_to_copy = ["Buffer", "Product", "PubMedID", "Publication", "pH", "Temperature", "Enzyme Variant", "UniProtKB_AC", "Organism", "KineticMechanismType", "SabioReactionID"]
                        for field in fields_to_copy:  
                            dict_entryid_to_append[field] = head_of_df[field]
                        dict_reactions_to_append[entryid] = dict_entryid_to_append
                        dict_entryid_to_append["Substrates"] = head_of_df["Substrate"].split(";")
                        out_rate_law = rate_law
                        if not rate_law in bad_rate_laws:                    
                            substrates = head_of_df["Substrate"].split(";")

                            stripped_string = re.sub('[0-9]', '', rate_law)

                            variables = re.split("\^|\*|\+|\-|\/|\(|\)| ", stripped_string)
                            variables = ' '.join(variables).split()

                            start_value_permutations = ["start value", "start val."]
                            substrates_key = {}
                            for var in variables:
                                if var in parameter_info:
                                    for permutation in start_value_permutations:
                                        try:
                                            if var == "A" or var == "B":
                                                substrates_key[var] = parameter_info[var]["species"]
                                            else:
                                                value = parameter_info[var][permutation]
                                                if value != "-" and value != "" and value != " ":
                                                    out_rate_law = out_rate_law.replace(var, parameter_info[var][permutation])
                                        except:
                                            pass

                            dict_entryid_to_append["RateLawSubstrates"] = substrates_key
                            dict_entryid_to_append["SubstitutedRateLaw"] = out_rate_law

                dict_to_append[reaction] = dict_reactions_to_append

            enzyme_dict[enzyme] = dict_to_append

        with open(scraped_model_json_file_path, 'w') as f:
            json.dump(enzyme_dict, f, indent=4)
        
        self.step_number = 5
        self.progress_update(self.step_number)
        
    def progress_update(self, step):
        if not re.search('[0-5]', str(step)):
            print(f'--> ERROR: The {step} step is not acceptable.')
        f = open(self.paths['progress_file_path'], "w")
        f.write(str(step))
        f.close()

    def main(self,):
        self.start()

        while True:
            if self.step_number == 1:
                self.scrape_bigg_xls()
            elif self.step_number == 2:
                self.glob_xls_files()
            elif self.step_number == 3:
                self.scrape_entryids()
            elif self.step_number == 4:
                self.combine_data()
            elif self.step_number == 5:
                print("Execution complete. Scraper finished.")
                break

In [ ]:
from selenium import webdriver
import os
fp = webdriver.FirefoxProfile("l2pnahxq.scraper")
fp.set_preference("browser.download.folderList",2)
fp.set_preference("browser.download.dir", os.getcwd())
driver = webdriver.Firefox(firefox_profile=fp, executable_path="geckodriver.exe")

# Chrome driver

In [3]:
# C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\BiGG_models\BiGG model of S. aureus.json
# ./BiGG_models/BiGG model of S. aureus.json

scraping = SABIO_scraping()
scraping.main()

Specify the BIGG Model JSON file path: ./BiGG_models/BiGG model of S. aureus.json
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\dFBApy\scraping\SABIO\BiGG_models


C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: DeprecationWarning: use options instead of chrome_options


ValueError: No objects to concatenate

# Newest

In [2]:
# -*- coding: utf-8 -*-
"""
@authors: Ethan Chan, Matthew Freiburger
"""
# Import libraries
import pandas as pd
import glob
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
from itertools import islice
import re



class SABIO_scraping():
#     __slots__ = (str(x) for x in [progress_file_prefix, xls_download_prefix, scraped_xls_prefix, scraped_entryids_prefix, sel_xls_download_path, processed_xls, entry_json, scraped_model, bigg_model_name_suffix, sub_directory_path, progress_file_path, xls_download_path, scraped_xls_file_path, scraped_entryids_file_path, xls_csv_file_path, entryids_json_file_path, scraped_model_json_file_path, bigg_model, step_number, cwd])
    
    def __init__(self,source='sabio'):
        driver = webdriver.Firefox(executable_path=r".\geckodriver.exe")
        if source == 'sabio':
            driver.get("http://sabiork.h-its.org/newSearch/index")
            self.source = 'sabio'
            
        self.count = 0
            
        self.parameters = {}
        self.parameters['general_delay'] = 2
        self.variables = {}
        self.paths = {}

    #Clicks a HTML element with selenium by id
    def click_element_id(self,n_id):
        element = self.driver.find_element_by_id(n_id)
        element.click()
        time.sleep(self.parameters['general_delay'])

    #Selects a choice from a HTML dropdown element with selenium by id
    def select_dropdown_id(self,n_id, n_choice):
        element = Select(self.driver.find_element_by_id(n_id))
        element.select_by_visible_text(n_choice)
        time.sleep(self.parameters['general_delay'])

    def fatal_error_handler(self,message):
        print("Error: " + message)
        print("Exiting now...")
        exit(0)

    """
    --------------------------------------------------------------------
        STEP 0: GET BIGG MODEL TO SCRAPE AND SETUP DIRECTORIES AND PROGRESS FILE
    --------------------------------------------------------------------    
    """

    def start(self,): # find the BiGG model that will be scraped
        while True:
            bigg_model_path = input("Specify the BIGG Model JSON file path: ")

            if os.path.exists(bigg_model_path) and bigg_model_path[-5:] == ".json":
                try:
                    self.model = json.load(open(bigg_model_path))
                    bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()
                    break
                except:
                    pass

#         self.model = json.load(open('BiGG model of S. aureus.json'))
#         bigg_model_path = 'BiGG model of S. aureus.json'
#         bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()

        # define the paths
        self.paths['cwd'] = os.path.dirname(os.path.realpath(bigg_model_path))
        print(self.paths['cwd'])
        self.paths['sub_directory_path'] = os.path.join(self.paths['cwd'],f"scraping-{bigg_model_name}")
        if not os.path.isdir(self.paths['sub_directory_path']):        
            os.mkdir(self.paths['sub_directory_path'])
                    
        self.variables['scraped_xls'] = {}
        self.variables['scraped_entryids'] = {}
        self.paths['scraped_model_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_model") + ".json"
        self.paths['sel_xls_download_path'] = os.path.join(self.paths['sub_directory_path'],"xls_download")
        
        self.step_number = 1
        
        self.paths['progress_file_path'] = os.path.join(self.paths['sub_directory_path'], "current_progress") + '.txt'
        if os.path.exists(self.paths['progress_file_path']):
            f = open(self.paths['progress_file_path'], "r")
            self.step_number = int(f.read(1))
            if not re.search('[1-5]',str(self.step_number)):
                self.fatal_error_handler("Progress file malformed. Please delete and restart")
        else:
            self.step_number = 1
            self.progress_update(self.step_number)

        self.paths['xls_download_path'] = os.path.join(self.paths['sub_directory_path'], 'downloaded_xls') 
        if not os.path.isdir(self.paths['xls_download_path']):
            os.mkdir(self.paths['xls_download_path'])

        self.paths['scraped_xls_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_xls") + ".json"
        if os.path.exists(self.paths['scraped_xls_file_path']):
            f = open(self.paths['scraped_xls_file_path'], "r")
            self.variables['scraped_xls'] = json.load(f)
            f.close()

        self.paths['scraped_entryids_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_entryids") + ".json"
        if os.path.exists(self.paths['scraped_entryids_file_path']):
            f = open(self.paths['scraped_entryids_file_path'], "r")
            self.variables['scraped_entryids'] = json.load(f)
            f.close()

        self.paths['entryids_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "entryids_json") + ".json"
        if os.path.exists(self.paths['entryids_json_file_path']):
            f = open(self.paths['entryids_json_file_path'], "r")
            self.variables['entry_id_json_out'] = json.load(f)
            f.close()

    """
    --------------------------------------------------------------------
        STEP 1: SCRAPE SABIO WEBSITE BY DOWNLOAD XLS FOR GIVEN REACTIONS IN BIGG MODEL
    --------------------------------------------------------------------    
    """

    def scrape_xls(self,reaction_identifier, search_option):
        self.count += 1
        print(self.count, end='\r')
        
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")
        self.click_element_id("resetbtn")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", search_option)
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(reaction_identifier)  
        time.sleep(self.parameters['general_delay'])  
        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        result_num = ""
        try: 
            result_num_ele = self.driver.find_element_by_id("numberofKinLaw")
            for char in result_num_ele.text:
                if re.search('[0-9]', char):
                    result_num = result_num + char

            result_num = int(result_num)
        except:
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        time.sleep(self.parameters['general_delay'])

        self.select_dropdown_id("max", "100")
        element = Select(self.driver.find_element_by_id("max"))
        element.select_by_visible_text("100")

        time.sleep(self.parameters['general_delay'])

        if result_num > 0 and result_num <= 100:
            self.click_element_id("allCheckbox")
            time.sleep(self.parameters['general_delay'])
        elif result_num > 100:
            self.click_element_id("allCheckbox")
            for i in range(int(result_num/100)):
                element = self.driver.find_element_by_xpath("//*[@class = 'nextLink']")
                element.click()
                time.sleep(self.parameters['general_delay'])
                self.click_element_id("allCheckbox")
                time.sleep(self.parameters['general_delay'])
        else:
            self.driver.get("http://sabiork.h-its.org/newSearch/index")
            return False

        self.driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")

        time.sleep(self.parameters['general_delay']*7.5)

        self.click_element_id("excelExport")

        time.sleep(self.parameters['general_delay']*2.5)

        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        return True
    
    def expand_shadow_element(self, element):
        shadow_root = self.driver.execute_script('return arguments[0].shadowRoot', element)
        return shadow_root


    def scrape_bigg_xls(self,):
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path']}
        chrome_options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)
        
        self.driver.get("chrome://settings/security")

        
        root = self.driver.find_element_by_tag_name("settings-ui")
        shadow_root = self.expand_shadow_element(root)
        
        root1 = shadow_root.find_element_by_tag_name("settings-main")
        shadow_root1 = self.expand_shadow_element(root1)
        
        root2 = shadow_root1.find_element_by_tag_name("settings-basic-page")
        shadow_root2 = self.expand_shadow_element(root2)
        
        root3 = shadow_root2.find_element_by_tag_name("settings-privacy-page")
        shadow_root3 = self.expand_shadow_element(root3)
        
        root4 = shadow_root3.find_element_by_tag_name("settings-security-page")
        shadow_root4 = self.expand_shadow_element(root4)
        
        root5 = shadow_root4.find_element_by_css_selector("#safeBrowsingStandard")
        shadow_root5 = self.expand_shadow_element(root5)
        
        security_button = shadow_root5.find_element_by_css_selector("div.disc-border")
        
        security_button.click()


        
        for reaction in self.model["reactions"]:
            if not reaction["name"] in self.variables['scraped_xls']:
                ids_to_try = reaction["annotation"]

                success_flag = False
                annotation_search_pairs = {"sabiork":"SabioReactionID", "metanetx.reaction":"MetaNetXReactionID", "ec-code":"ECNumber", "kegg.reaction":"KeggReactionID", "rhea":"RheaReactionID"}
                for annotation in annotation_search_pairs:
                    if not success_flag:
                        if annotation in ids_to_try:


                            for id_to_try in ids_to_try[annotation]:
#                                 success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                try:
                                    success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                except:
                                    success_flag = False
                    else:
                        break

                if not success_flag:
                    try:
                        success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    except:
                        success_flag = False

                json_dict_key = reaction["name"].replace("\"", "")
                if success_flag:
                    self.variables['scraped_xls'][json_dict_key] = "yes"
                else:
                    self.variables['scraped_xls'][json_dict_key] = "no"
                    
                    

            with open(self.paths['scraped_xls_file_path'], 'w') as outfile:
                json.dump(self.variables['scraped_xls'], outfile, indent = 4)   
                outfile.close()
                
        self.step_number = 2
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 2: GLOB EXPORTED XLS FILES TOGETHER
    --------------------------------------------------------------------
    """

    def glob_xls_files(self,):
        scraped_sans_parentheses_enzymes = glob.glob('./{}/*.xls'.format(self.paths['xls_download_path']))
        total_dataframes = []
        for file in scraped_sans_parentheses_enzymes:
            #file_name = os.path.splitext(os.path.basename(file))[0]
            dfn = pd.read_excel(file)
            total_dataframes.append(dfn)

        # combine the total set of dataframes
        combined_df = pd.DataFrame()
        combined_df = pd.concat(total_dataframes)
        combined_df = combined_df.fillna(' ')
        combined_df = combined_df.drop_duplicates()

        # export the dataframe
        csv_path = os.path.join(self.paths['sub_directory_path'], "proccessed-xls") + ".csv"
        combined_df.to_csv(csv_path)
        
        # update the 
        self.step_number = 3
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 3: SCRAPE ADDITIONAL DATA BY ENTRYID
    --------------------------------------------------------------------    
    """

    def scrape_entry_id(self,entry_id):
        global entry_id_json_out

        entry_id = str(entry_id)

        self.driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(general_delay)

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", "EntryID")
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(entry_id)

        time.sleep(general_delay)

        self.click_element_id("addsearch")

        time.sleep(general_delay)

        self.click_element_id(entry_id + "img")

        time.sleep(general_delay)

        self.driver.switch_to.frame(self.driver.find_element_by_xpath("//iframe[@name='iframe_" + entry_id + "']"))
        element = self.driver.find_element_by_xpath("//table")
        html_source = element.get_attribute('innerHTML')
        
        table_df = pd.read_html(html_source)
        reaction_parameters_df = pd.DataFrame()
        counter = 0
        parameters_json = {}
        for df in table_df:
            try:
                if df[0][0] == "Parameter":
                    reaction_parameters_df = table_df[counter]
            except:
                self.driver.close()
                return parameters_json
            counter += 1
            
        parameter_name = ""
        for i in range(len(reaction_parameters_df[0])-2):
            parameter_name = reaction_parameters_df[0][i+2]
            inner_parameters_json = {}
            for j in range(len(reaction_parameters_df)-3):
                inner_parameters_json[reaction_parameters_df[j+1][1]] = reaction_parameters_df[j+1][i+2]

            parameters_json[parameter_name] = inner_parameters_json

        self.driver.close()

        return parameters_json


    def scrape_entryids(self,):
        global entry_id_json_out

        sabio_xls_df = pd.read_csv(xls_csv_file_path)
        entryids = sabio_xls_df["EntryID"].unique().tolist()

        for entryid in entryids:
            if not entryid in scraped_entryids:
                try:
                    entry_id_json_out[str(entryid)] = scrape_entry_id(entryid)
                    self.variables['scraped_entryids'][entryid] = "yes"
                except:
                    self.variables['scraped_entryids'][entryid] = "no"
            with open(scraped_entryids_file_path, 'w') as outfile:
                json.dump(self.variables['scraped_entryids'], outfile, indent = 4)   
                outfile.close()
            with open(entryids_json_file_path, 'w') as f:
                json.dump(entry_id_json_out, f, indent = 4)        
                f.close()
        
        self.step_number = 4
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 4: COMBINE ENZYME AND ENTRYID DATA INTO JSON FILE
    --------------------------------------------------------------------
    """

    def combine_data(self,):

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])

        # Opening JSON file
        with open(self.paths['entryids_json_file_path']) as json_file:
            entry_id_data = json.load(json_file)

        enzymenames = sabio_xls_df["Enzymename"].unique().tolist()
        enzyme_dict = {}
        missing_entry_ids = []
        parameters = {}

        for enzyme in enzymenames:
            sabio_grouped_enzyme_df = sabio_xls_df.loc[sabio_xls_df["Enzymename"] == enzyme]
            dict_to_append = {}
            reactions = sabio_grouped_enzyme_df["Reaction"].unique().tolist()
            for reaction in reactions:
                dict_reactions_to_append = {}
                sabio_grouped_reactions_df = sabio_grouped_enzyme_df.loc[sabio_grouped_enzyme_df["Reaction"] == reaction]
                entryids = sabio_grouped_reactions_df["EntryID"].unique().tolist()

                for entryid in entryids:
                    entry_ids_df = sabio_grouped_reactions_df.loc[sabio_grouped_reactions_df["EntryID"] == entryid]
                    dict_entryid_to_append = {}
                    head_of_df = entry_ids_df.head(1).squeeze()
                    entry_id_flag = True
                    parameter_info = {}

                    try:
                        parameter_info = entry_id_data[str(entryid)]
                        dict_entryid_to_append["Parameters"] = parameter_info
                    except:
                        missing_entry_ids.append(str(entryid))
                        entry_id_flag = False
                        dict_entryid_to_append["Parameters"] = "NaN"

                    rate_law = head_of_df["Rate Equation"]
                    bad_rate_laws = ["unknown", "", "-"]

                    if not rate_law in bad_rate_laws:                    
                        dict_entryid_to_append["RateLaw"] = rate_law
                        dict_entryid_to_append["SubstitutedRateLaw"] = rate_law
                    else:
                        dict_entryid_to_append["RateLaw"] = "NaN"
                        dict_entryid_to_append["SubstitutedRateLaw"] = "NaN"

                    if entry_id_flag:

                        fields_to_copy = ["Buffer", "Product", "PubMedID", "Publication", "pH", "Temperature", "Enzyme Variant", "UniProtKB_AC", "Organism", "KineticMechanismType", "SabioReactionID"]
                        for field in fields_to_copy:  
                            dict_entryid_to_append[field] = head_of_df[field]
                        dict_reactions_to_append[entryid] = dict_entryid_to_append
                        dict_entryid_to_append["Substrates"] = head_of_df["Substrate"].split(";")
                        out_rate_law = rate_law
                        if not rate_law in bad_rate_laws:                    
                            substrates = head_of_df["Substrate"].split(";")

                            stripped_string = re.sub('[0-9]', '', rate_law)

                            variables = re.split("\^|\*|\+|\-|\/|\(|\)| ", stripped_string)
                            variables = ' '.join(variables).split()

                            start_value_permutations = ["start value", "start val."]
                            substrates_key = {}
                            for var in variables:
                                if var in parameter_info:
                                    for permutation in start_value_permutations:
                                        try:
                                            if var == "A" or var == "B":
                                                substrates_key[var] = parameter_info[var]["species"]
                                            else:
                                                value = parameter_info[var][permutation]
                                                if value != "-" and value != "" and value != " ":
                                                    out_rate_law = out_rate_law.replace(var, parameter_info[var][permutation])
                                        except:
                                            pass

                            dict_entryid_to_append["RateLawSubstrates"] = substrates_key
                            dict_entryid_to_append["SubstitutedRateLaw"] = out_rate_law

                dict_to_append[reaction] = dict_reactions_to_append

            enzyme_dict[enzyme] = dict_to_append

        with open(scraped_model_json_file_path, 'w') as f:
            json.dump(enzyme_dict, f, indent=4)
        
        self.step_number = 5
        self.progress_update(self.step_number)
        
    def progress_update(self, step):
        if not re.search('[0-5]', str(step)):
            print(f'--> ERROR: The {step} step is not acceptable.')
        f = open(self.paths['progress_file_path'], "w")
        f.write(str(step))
        f.close()

    def main(self,):
        self.start()

        while True:
            if self.step_number == 1:
                self.scrape_bigg_xls()
            elif self.step_number == 2:
                self.glob_xls_files()
            elif self.step_number == 3:
                self.scrape_entryids()
            elif self.step_number == 4:
                self.combine_data()
            elif self.step_number == 5:
                print("Execution complete. Scraper finished.")
                break

In [4]:
# -*- coding: utf-8 -*-
"""
@authors: Ethan Chan, Matthew Freiburger
"""
# Import libraries
import pandas as pd
import glob
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
from itertools import islice
import re



class SABIO_scraping():
#     __slots__ = (str(x) for x in [progress_file_prefix, xls_download_prefix, scraped_xls_prefix, scraped_entryids_prefix, sel_xls_download_path, processed_xls, entry_json, scraped_model, bigg_model_name_suffix, sub_directory_path, progress_file_path, xls_download_path, scraped_xls_file_path, scraped_entryids_file_path, xls_csv_file_path, entryids_json_file_path, scraped_model_json_file_path, bigg_model, step_number, cwd])
    
    def __init__(self,source='sabio'):
        driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        if source == 'sabio':
            driver.get("http://sabiork.h-its.org/newSearch/index")
            self.source = 'sabio'
            
        self.count = 0
            
        self.parameters = {}
        self.parameters['general_delay'] = 2
        self.variables = {}
        self.paths = {}

    #Clicks a HTML element with selenium by id
    def click_element_id(self,n_id):
        element = self.driver.find_element_by_id(n_id)
        element.click()
        time.sleep(self.parameters['general_delay'])

    #Selects a choice from a HTML dropdown element with selenium by id
    def select_dropdown_id(self,n_id, n_choice):
        element = Select(self.driver.find_element_by_id(n_id))
        element.select_by_visible_text(n_choice)
        time.sleep(self.parameters['general_delay'])

    def fatal_error_handler(self,message):
        print("Error: " + message)
        print("Exiting now...")
        exit(0)

    """
    --------------------------------------------------------------------
        STEP 0: GET BIGG MODEL TO SCRAPE AND SETUP DIRECTORIES AND PROGRESS FILE
    --------------------------------------------------------------------    
    """

    def start(self,): # find the BiGG model that will be scraped
        while True:
            bigg_model_path = input("Specify the BIGG Model JSON file path: ")

            if os.path.exists(bigg_model_path) and bigg_model_path[-5:] == ".json":
                try:
                    self.model = json.load(open(bigg_model_path))
                    bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()
                    break
                except:
                    pass

#         self.model = json.load(open('BiGG model of S. aureus.json'))
#         bigg_model_path = 'BiGG model of S. aureus.json'
#         bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()

        # define the paths
        self.paths['cwd'] = os.path.dirname(os.path.realpath(bigg_model_path))
        print(self.paths['cwd'])
        self.paths['sub_directory_path'] = os.path.join(self.paths['cwd'],f"scraping-{bigg_model_name}")
        if not os.path.isdir(self.paths['sub_directory_path']):        
            os.mkdir(self.paths['sub_directory_path'])
                    
        self.variables['scraped_xls'] = {}
        self.variables['scraped_entryids'] = {}
        self.paths['scraped_model_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_model") + ".json"
        self.paths['sel_xls_download_path'] = os.path.join(self.paths['sub_directory_path'],"xls_download")
        
        self.step_number = 1
        
        self.paths['progress_file_path'] = os.path.join(self.paths['sub_directory_path'], "current_progress") + '.txt'
        if os.path.exists(self.paths['progress_file_path']):
            f = open(self.paths['progress_file_path'], "r")
            self.step_number = int(f.read(1))
            if not re.search('[1-5]',str(self.step_number)):
                self.fatal_error_handler("Progress file malformed. Please delete and restart")
        else:
            self.step_number = 1
            self.progress_update(self.step_number)

        self.paths['xls_download_path'] = os.path.join(self.paths['sub_directory_path'], 'downloaded_xls') 
        if not os.path.isdir(self.paths['xls_download_path']):
            os.mkdir(self.paths['xls_download_path'])

        self.paths['scraped_xls_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_xls") + ".json"
        if os.path.exists(self.paths['scraped_xls_file_path']):
            f = open(self.paths['scraped_xls_file_path'], "r")
            self.variables['scraped_xls'] = json.load(f)
            f.close()

        self.paths['scraped_entryids_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_entryids") + ".json"
        if os.path.exists(self.paths['scraped_entryids_file_path']):
            f = open(self.paths['scraped_entryids_file_path'], "r")
            self.variables['scraped_entryids'] = json.load(f)
            f.close()

        self.paths['entryids_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "entryids_json") + ".json"
        if os.path.exists(self.paths['entryids_json_file_path']):
            f = open(self.paths['entryids_json_file_path'], "r")
            self.variables['entry_id_json_out'] = json.load(f)
            f.close()

    """
    --------------------------------------------------------------------
        STEP 1: SCRAPE SABIO WEBSITE BY DOWNLOAD XLS FOR GIVEN REACTIONS IN BIGG MODEL
    --------------------------------------------------------------------    
    """

    def scrape_xls(self,reaction_identifier, search_option):
        self.count += 1
        print(self.count, end='\r')
        
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path'], "download.prompt_for_download": False, "download.directory_upgrade": True, "safebrowsing.enabled": True}

#         prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path']}
        chrome_options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)    
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", search_option)
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(reaction_identifier)  
        time.sleep(self.parameters['general_delay'])  
        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        result_num = ""
        try: 
            result_num_ele = self.driver.find_element_by_id("numberofKinLaw")
            for char in result_num_ele.text:
                if re.search('[0-9]', char):
                    result_num = result_num + char

            result_num = int(result_num)
        except:
            self.driver.close()
            return False

        time.sleep(self.parameters['general_delay'])

        self.select_dropdown_id("max", "100")
        element = Select(self.driver.find_element_by_id("max"))
        element.select_by_visible_text("100")

        time.sleep(self.parameters['general_delay'])

        if result_num > 0 and result_num <= 100:
            self.click_element_id("allCheckbox")
            time.sleep(self.parameters['general_delay'])
        elif result_num > 100:
            self.click_element_id("allCheckbox")
            for i in range(int(result_num/100)):
                element = self.driver.find_element_by_xpath("//*[@class = 'nextLink']")
                element.click()
                time.sleep(self.parameters['general_delay'])
                self.click_element_id("allCheckbox")
                time.sleep(self.parameters['general_delay'])
        else:
            self.driver.close()
            return False

        self.driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")

        time.sleep(self.parameters['general_delay']*7.5)

        self.click_element_id("excelExport")

        time.sleep(self.parameters['general_delay']*2.5)

        self.driver.close()

        return True


    def scrape_bigg_xls(self,):
        for reaction in self.model["reactions"]:
            if not reaction["name"] in self.variables['scraped_xls']:
                ids_to_try = reaction["annotation"]

                success_flag = False
                annotation_search_pairs = {"sabiork":"SabioReactionID", "metanetx.reaction":"MetaNetXReactionID", "ec-code":"ECNumber", "kegg.reaction":"KeggReactionID", "rhea":"RheaReactionID"}
                for annotation in annotation_search_pairs:
                    if not success_flag:
                        if annotation in ids_to_try:
                            for id_to_try in ids_to_try[annotation]:
#                                 success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                try:
                                    success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                except:
                                    success_flag = False
                    else:
                        break

                if not success_flag:
#                     success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    try:
                        success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    except:
                        success_flag = False

                json_dict_key = reaction["name"].replace("\"", "")
                if success_flag:
                    self.variables['scraped_xls'][json_dict_key] = "yes"
                else:
                    self.variables['scraped_xls'][json_dict_key] = "no"

            with open(self.paths['scraped_xls_file_path'], 'w') as outfile:
                json.dump(self.variables['scraped_xls'], outfile, indent = 4)   
                outfile.close()
                
        self.step_number = 2
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 2: GLOB EXPORTED XLS FILES TOGETHER
    --------------------------------------------------------------------
    """

    def glob_xls_files(self,):
        scraped_sans_parentheses_enzymes = glob.glob('./{}/*.xls'.format(self.paths['xls_download_path']))
        total_dataframes = []
        for file in scraped_sans_parentheses_enzymes:
            #file_name = os.path.splitext(os.path.basename(file))[0]
            dfn = pd.read_excel(file)
            total_dataframes.append(dfn)

        # combine the total set of dataframes
        combined_df = pd.DataFrame()
        combined_df = pd.concat(total_dataframes)
        combined_df = combined_df.fillna(' ')
        combined_df = combined_df.drop_duplicates()

        # export the dataframe
        csv_path = os.path.join(self.paths['sub_directory_path'], "proccessed-xls") + ".csv"
        combined_df.to_csv(csv_path)
        
        # update the 
        self.step_number = 3
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 3: SCRAPE ADDITIONAL DATA BY ENTRYID
    --------------------------------------------------------------------    
    """

    def scrape_entry_id(self,entry_id):
        global entry_id_json_out

        entry_id = str(entry_id)

        self.driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(general_delay)

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", "EntryID")
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(entry_id)

        time.sleep(general_delay)

        self.click_element_id("addsearch")

        time.sleep(general_delay)

        self.click_element_id(entry_id + "img")

        time.sleep(general_delay)

        self.driver.switch_to.frame(self.driver.find_element_by_xpath("//iframe[@name='iframe_" + entry_id + "']"))
        element = self.driver.find_element_by_xpath("//table")
        html_source = element.get_attribute('innerHTML')
        
        table_df = pd.read_html(html_source)
        reaction_parameters_df = pd.DataFrame()
        counter = 0
        parameters_json = {}
        for df in table_df:
            try:
                if df[0][0] == "Parameter":
                    reaction_parameters_df = table_df[counter]
            except:
                self.driver.close()
                return parameters_json
            counter += 1
            
        parameter_name = ""
        for i in range(len(reaction_parameters_df[0])-2):
            parameter_name = reaction_parameters_df[0][i+2]
            inner_parameters_json = {}
            for j in range(len(reaction_parameters_df)-3):
                inner_parameters_json[reaction_parameters_df[j+1][1]] = reaction_parameters_df[j+1][i+2]

            parameters_json[parameter_name] = inner_parameters_json

        self.driver.close()

        return parameters_json


    def scrape_entryids(self,):
        global entry_id_json_out

        sabio_xls_df = pd.read_csv(xls_csv_file_path)
        entryids = sabio_xls_df["EntryID"].unique().tolist()

        for entryid in entryids:
            if not entryid in scraped_entryids:
                try:
                    entry_id_json_out[str(entryid)] = scrape_entry_id(entryid)
                    self.variables['scraped_entryids'][entryid] = "yes"
                except:
                    self.variables['scraped_entryids'][entryid] = "no"
            with open(scraped_entryids_file_path, 'w') as outfile:
                json.dump(self.variables['scraped_entryids'], outfile, indent = 4)   
                outfile.close()
            with open(entryids_json_file_path, 'w') as f:
                json.dump(entry_id_json_out, f, indent = 4)        
                f.close()
        
        self.step_number = 4
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 4: COMBINE ENZYME AND ENTRYID DATA INTO JSON FILE
    --------------------------------------------------------------------
    """

    def combine_data(self,):

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])

        # Opening JSON file
        with open(self.paths['entryids_json_file_path']) as json_file:
            entry_id_data = json.load(json_file)

        enzymenames = sabio_xls_df["Enzymename"].unique().tolist()
        enzyme_dict = {}
        missing_entry_ids = []
        parameters = {}

        for enzyme in enzymenames:
            sabio_grouped_enzyme_df = sabio_xls_df.loc[sabio_xls_df["Enzymename"] == enzyme]
            dict_to_append = {}
            reactions = sabio_grouped_enzyme_df["Reaction"].unique().tolist()
            for reaction in reactions:
                dict_reactions_to_append = {}
                sabio_grouped_reactions_df = sabio_grouped_enzyme_df.loc[sabio_grouped_enzyme_df["Reaction"] == reaction]
                entryids = sabio_grouped_reactions_df["EntryID"].unique().tolist()

                for entryid in entryids:
                    entry_ids_df = sabio_grouped_reactions_df.loc[sabio_grouped_reactions_df["EntryID"] == entryid]
                    dict_entryid_to_append = {}
                    head_of_df = entry_ids_df.head(1).squeeze()
                    entry_id_flag = True
                    parameter_info = {}

                    try:
                        parameter_info = entry_id_data[str(entryid)]
                        dict_entryid_to_append["Parameters"] = parameter_info
                    except:
                        missing_entry_ids.append(str(entryid))
                        entry_id_flag = False
                        dict_entryid_to_append["Parameters"] = "NaN"

                    rate_law = head_of_df["Rate Equation"]
                    bad_rate_laws = ["unknown", "", "-"]

                    if not rate_law in bad_rate_laws:                    
                        dict_entryid_to_append["RateLaw"] = rate_law
                        dict_entryid_to_append["SubstitutedRateLaw"] = rate_law
                    else:
                        dict_entryid_to_append["RateLaw"] = "NaN"
                        dict_entryid_to_append["SubstitutedRateLaw"] = "NaN"

                    if entry_id_flag:

                        fields_to_copy = ["Buffer", "Product", "PubMedID", "Publication", "pH", "Temperature", "Enzyme Variant", "UniProtKB_AC", "Organism", "KineticMechanismType", "SabioReactionID"]
                        for field in fields_to_copy:  
                            dict_entryid_to_append[field] = head_of_df[field]
                        dict_reactions_to_append[entryid] = dict_entryid_to_append
                        dict_entryid_to_append["Substrates"] = head_of_df["Substrate"].split(";")
                        out_rate_law = rate_law
                        if not rate_law in bad_rate_laws:                    
                            substrates = head_of_df["Substrate"].split(";")

                            stripped_string = re.sub('[0-9]', '', rate_law)

                            variables = re.split("\^|\*|\+|\-|\/|\(|\)| ", stripped_string)
                            variables = ' '.join(variables).split()

                            start_value_permutations = ["start value", "start val."]
                            substrates_key = {}
                            for var in variables:
                                if var in parameter_info:
                                    for permutation in start_value_permutations:
                                        try:
                                            if var == "A" or var == "B":
                                                substrates_key[var] = parameter_info[var]["species"]
                                            else:
                                                value = parameter_info[var][permutation]
                                                if value != "-" and value != "" and value != " ":
                                                    out_rate_law = out_rate_law.replace(var, parameter_info[var][permutation])
                                        except:
                                            pass

                            dict_entryid_to_append["RateLawSubstrates"] = substrates_key
                            dict_entryid_to_append["SubstitutedRateLaw"] = out_rate_law

                dict_to_append[reaction] = dict_reactions_to_append

            enzyme_dict[enzyme] = dict_to_append

        with open(scraped_model_json_file_path, 'w') as f:
            json.dump(enzyme_dict, f, indent=4)
        
        self.step_number = 5
        self.progress_update(self.step_number)
        
    def progress_update(self, step):
        if not re.search('[0-5]', str(step)):
            print(f'--> ERROR: The {step} step is not acceptable.')
        f = open(self.paths['progress_file_path'], "w")
        f.write(str(step))
        f.close()

    def main(self,):
        self.start()

        while True:
            if self.step_number == 1:
                self.scrape_bigg_xls()
            elif self.step_number == 2:
                self.glob_xls_files()
            elif self.step_number == 3:
                self.scrape_entryids()
            elif self.step_number == 4:
                self.combine_data()
            elif self.step_number == 5:
                print("Execution complete. Scraper finished.")
                break

In [6]:
# -*- coding: utf-8 -*-
"""
@authors: Ethan Chan, Matthew Freiburger
"""
# Import libraries
import pandas as pd
import glob
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
from itertools import islice
import re



class SABIO_scraping():
#     __slots__ = (str(x) for x in [progress_file_prefix, xls_download_prefix, scraped_xls_prefix, scraped_entryids_prefix, sel_xls_download_path, processed_xls, entry_json, scraped_model, bigg_model_name_suffix, sub_directory_path, progress_file_path, xls_download_path, scraped_xls_file_path, scraped_entryids_file_path, xls_csv_file_path, entryids_json_file_path, scraped_model_json_file_path, bigg_model, step_number, cwd])
    
    def __init__(self,source='sabio'):
        driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        if source == 'sabio':
            driver.get("http://sabiork.h-its.org/newSearch/index")
            self.source = 'sabio'
            
        self.count = 0
            
        self.parameters = {}
        self.parameters['general_delay'] = 2
        self.variables = {}
        self.paths = {}

    #Clicks a HTML element with selenium by id
    def click_element_id(self,n_id):
        element = self.driver.find_element_by_id(n_id)
        element.click()
        time.sleep(self.parameters['general_delay'])

    #Selects a choice from a HTML dropdown element with selenium by id
    def select_dropdown_id(self,n_id, n_choice):
        element = Select(self.driver.find_element_by_id(n_id))
        element.select_by_visible_text(n_choice)
        time.sleep(self.parameters['general_delay'])

    def fatal_error_handler(self,message):
        print("Error: " + message)
        print("Exiting now...")
        exit(0)

    """
    --------------------------------------------------------------------
        STEP 0: GET BIGG MODEL TO SCRAPE AND SETUP DIRECTORIES AND PROGRESS FILE
    --------------------------------------------------------------------    
    """

    def start(self,): # find the BiGG model that will be scraped
        while True:
            bigg_model_path = input("Specify the BIGG Model JSON file path: ")

            if os.path.exists(bigg_model_path) and bigg_model_path[-5:] == ".json":
                try:
                    self.model = json.load(open(bigg_model_path))
                    bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()
                    break
                except:
                    pass

#         self.model = json.load(open('BiGG model of S. aureus.json'))
#         bigg_model_path = 'BiGG model of S. aureus.json'
#         bigg_model_name = re.search("([\w+\.?\s?]+)(?=\.json)", bigg_model_path).group()

        # define the paths
        self.paths['cwd'] = os.path.dirname(os.path.realpath(bigg_model_path))
        print(self.paths['cwd'])
        self.paths['sub_directory_path'] = os.path.join(self.paths['cwd'],f"scraping-{bigg_model_name}")
        if not os.path.isdir(self.paths['sub_directory_path']):        
            os.mkdir(self.paths['sub_directory_path'])
                    
        self.variables['scraped_xls'] = {}
        self.variables['scraped_entryids'] = {}
        self.paths['scraped_model_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_model") + ".json"
        self.paths['sel_xls_download_path'] = os.path.join(self.paths['sub_directory_path'],"xls_download")
        
        self.step_number = 1
        
        self.paths['progress_file_path'] = os.path.join(self.paths['sub_directory_path'], "current_progress") + '.txt'
        if os.path.exists(self.paths['progress_file_path']):
            f = open(self.paths['progress_file_path'], "r")
            self.step_number = int(f.read(1))
            if not re.search('[1-5]',str(self.step_number)):
                self.fatal_error_handler("Progress file malformed. Please delete and restart")
        else:
            self.step_number = 1
            self.progress_update(self.step_number)

        self.paths['xls_download_path'] = os.path.join(self.paths['sub_directory_path'], 'downloaded_xls') 
        if not os.path.isdir(self.paths['xls_download_path']):
            os.mkdir(self.paths['xls_download_path'])

        self.paths['scraped_xls_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_xls") + ".json"
        if os.path.exists(self.paths['scraped_xls_file_path']):
            f = open(self.paths['scraped_xls_file_path'], "r")
            self.variables['scraped_xls'] = json.load(f)
            f.close()

        self.paths['scraped_entryids_file_path'] = os.path.join(self.paths['sub_directory_path'], "scraped_entryids") + ".json"
        if os.path.exists(self.paths['scraped_entryids_file_path']):
            f = open(self.paths['scraped_entryids_file_path'], "r")
            self.variables['scraped_entryids'] = json.load(f)
            f.close()

        self.paths['entryids_json_file_path'] = os.path.join(self.paths['sub_directory_path'], "entryids_json") + ".json"
        if os.path.exists(self.paths['entryids_json_file_path']):
            f = open(self.paths['entryids_json_file_path'], "r")
            self.variables['entry_id_json_out'] = json.load(f)
            f.close()

    """
    --------------------------------------------------------------------
        STEP 1: SCRAPE SABIO WEBSITE BY DOWNLOAD XLS FOR GIVEN REACTIONS IN BIGG MODEL
    --------------------------------------------------------------------    
    """

    def scrape_xls(self,reaction_identifier, search_option):
        self.count += 1
        print(self.count, end='\r')
        
        
        self.driver.get("http://sabiork.h-its.org/newSearch/index")
        self.click_element_id("resetbtn")

        time.sleep(self.parameters['general_delay'])

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", search_option)
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(reaction_identifier)  
        time.sleep(self.parameters['general_delay'])  
        self.click_element_id("addsearch")

        time.sleep(self.parameters['general_delay'])

        result_num = ""
        try: 
            result_num_ele = self.driver.find_element_by_id("numberofKinLaw")
            for char in result_num_ele.text:
                if re.search('[0-9]', char):
                    result_num = result_num + char

            result_num = int(result_num)
        except:
            self.driver.close()
            return False

        time.sleep(self.parameters['general_delay'])

        self.select_dropdown_id("max", "100")
        element = Select(self.driver.find_element_by_id("max"))
        element.select_by_visible_text("100")

        time.sleep(self.parameters['general_delay'])

        if result_num > 0 and result_num <= 100:
            self.click_element_id("allCheckbox")
            time.sleep(self.parameters['general_delay'])
        elif result_num > 100:
            self.click_element_id("allCheckbox")
            for i in range(int(result_num/100)):
                element = self.driver.find_element_by_xpath("//*[@class = 'nextLink']")
                element.click()
                time.sleep(self.parameters['general_delay'])
                self.click_element_id("allCheckbox")
                time.sleep(self.parameters['general_delay'])
        else:
            #self.driver.close()
            return False

        self.driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")

        time.sleep(self.parameters['general_delay']*7.5)

        self.click_element_id("excelExport")

        time.sleep(self.parameters['general_delay']*2.5)

        #self.driver.close()

        return True


    def scrape_bigg_xls(self,):
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory' : self.paths['cwd'] + self.paths['sel_xls_download_path']}
        chrome_options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)
        
        self.driver.get("chrome://settings/security?search=downloads")
        element = self.driver.find_element_by_xpath("//div[contains(@class, 'disc-border')]")
        element.click()
        
        for reaction in self.model["reactions"]:
            if not reaction["name"] in self.variables['scraped_xls']:
                ids_to_try = reaction["annotation"]

                success_flag = False
                annotation_search_pairs = {"sabiork":"SabioReactionID", "metanetx.reaction":"MetaNetXReactionID", "ec-code":"ECNumber", "kegg.reaction":"KeggReactionID", "rhea":"RheaReactionID"}
                for annotation in annotation_search_pairs:
                    if not success_flag:
                        if annotation in ids_to_try:
                            for id_to_try in ids_to_try[annotation]:
#                                 success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                try:
                                    success_flag = self.scrape_xls(id_to_try, annotation_search_pairs[annotation])
                                except:
                                    success_flag = False
                    else:
                        break

                if not success_flag:
#                     success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    try:
                        success_flag = self.scrape_xls(reaction["name"], "Enzymename")
                    except:
                        success_flag = False

                json_dict_key = reaction["name"].replace("\"", "")
                if success_flag:
                    self.variables['scraped_xls'][json_dict_key] = "yes"
                else:
                    self.variables['scraped_xls'][json_dict_key] = "no"

            with open(self.paths['scraped_xls_file_path'], 'w') as outfile:
                json.dump(self.variables['scraped_xls'], outfile, indent = 4)   
                outfile.close()
                
        self.step_number = 2
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 2: GLOB EXPORTED XLS FILES TOGETHER
    --------------------------------------------------------------------
    """

    def glob_xls_files(self,):
        scraped_sans_parentheses_enzymes = glob.glob('./{}/*.xls'.format(self.paths['xls_download_path']))
        total_dataframes = []
        for file in scraped_sans_parentheses_enzymes:
            #file_name = os.path.splitext(os.path.basename(file))[0]
            dfn = pd.read_excel(file)
            total_dataframes.append(dfn)

        # combine the total set of dataframes
        combined_df = pd.DataFrame()
        combined_df = pd.concat(total_dataframes)
        combined_df = combined_df.fillna(' ')
        combined_df = combined_df.drop_duplicates()

        # export the dataframe
        csv_path = os.path.join(self.paths['sub_directory_path'], "proccessed-xls") + ".csv"
        combined_df.to_csv(csv_path)
        
        # update the 
        self.step_number = 3
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 3: SCRAPE ADDITIONAL DATA BY ENTRYID
    --------------------------------------------------------------------    
    """

    def scrape_entry_id(self,entry_id):
        global entry_id_json_out

        entry_id = str(entry_id)

        self.driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
        self.driver.get("http://sabiork.h-its.org/newSearch/index")

        time.sleep(general_delay)

        self.click_element_id("option")
        self.select_dropdown_id("searchterms", "EntryID")
        text_area = self.driver.find_element_by_id("searchtermField")
        text_area.send_keys(entry_id)

        time.sleep(general_delay)

        self.click_element_id("addsearch")

        time.sleep(general_delay)

        self.click_element_id(entry_id + "img")

        time.sleep(general_delay)

        self.driver.switch_to.frame(self.driver.find_element_by_xpath("//iframe[@name='iframe_" + entry_id + "']"))
        element = self.driver.find_element_by_xpath("//table")
        html_source = element.get_attribute('innerHTML')
        
        table_df = pd.read_html(html_source)
        reaction_parameters_df = pd.DataFrame()
        counter = 0
        parameters_json = {}
        for df in table_df:
            try:
                if df[0][0] == "Parameter":
                    reaction_parameters_df = table_df[counter]
            except:
                self.driver.close()
                return parameters_json
            counter += 1
            
        parameter_name = ""
        for i in range(len(reaction_parameters_df[0])-2):
            parameter_name = reaction_parameters_df[0][i+2]
            inner_parameters_json = {}
            for j in range(len(reaction_parameters_df)-3):
                inner_parameters_json[reaction_parameters_df[j+1][1]] = reaction_parameters_df[j+1][i+2]

            parameters_json[parameter_name] = inner_parameters_json

        self.driver.close()

        return parameters_json


    def scrape_entryids(self,):
        global entry_id_json_out

        sabio_xls_df = pd.read_csv(xls_csv_file_path)
        entryids = sabio_xls_df["EntryID"].unique().tolist()

        for entryid in entryids:
            if not entryid in scraped_entryids:
                try:
                    entry_id_json_out[str(entryid)] = scrape_entry_id(entryid)
                    self.variables['scraped_entryids'][entryid] = "yes"
                except:
                    self.variables['scraped_entryids'][entryid] = "no"
            with open(scraped_entryids_file_path, 'w') as outfile:
                json.dump(self.variables['scraped_entryids'], outfile, indent = 4)   
                outfile.close()
            with open(entryids_json_file_path, 'w') as f:
                json.dump(entry_id_json_out, f, indent = 4)        
                f.close()
        
        self.step_number = 4
        self.progress_update(self.step_number)

    """
    --------------------------------------------------------------------
        STEP 4: COMBINE ENZYME AND ENTRYID DATA INTO JSON FILE
    --------------------------------------------------------------------
    """

    def combine_data(self,):

        sabio_xls_df = pd.read_csv(self.paths['xls_csv_file_path'])

        # Opening JSON file
        with open(self.paths['entryids_json_file_path']) as json_file:
            entry_id_data = json.load(json_file)

        enzymenames = sabio_xls_df["Enzymename"].unique().tolist()
        enzyme_dict = {}
        missing_entry_ids = []
        parameters = {}

        for enzyme in enzymenames:
            sabio_grouped_enzyme_df = sabio_xls_df.loc[sabio_xls_df["Enzymename"] == enzyme]
            dict_to_append = {}
            reactions = sabio_grouped_enzyme_df["Reaction"].unique().tolist()
            for reaction in reactions:
                dict_reactions_to_append = {}
                sabio_grouped_reactions_df = sabio_grouped_enzyme_df.loc[sabio_grouped_enzyme_df["Reaction"] == reaction]
                entryids = sabio_grouped_reactions_df["EntryID"].unique().tolist()

                for entryid in entryids:
                    entry_ids_df = sabio_grouped_reactions_df.loc[sabio_grouped_reactions_df["EntryID"] == entryid]
                    dict_entryid_to_append = {}
                    head_of_df = entry_ids_df.head(1).squeeze()
                    entry_id_flag = True
                    parameter_info = {}

                    try:
                        parameter_info = entry_id_data[str(entryid)]
                        dict_entryid_to_append["Parameters"] = parameter_info
                    except:
                        missing_entry_ids.append(str(entryid))
                        entry_id_flag = False
                        dict_entryid_to_append["Parameters"] = "NaN"

                    rate_law = head_of_df["Rate Equation"]
                    bad_rate_laws = ["unknown", "", "-"]

                    if not rate_law in bad_rate_laws:                    
                        dict_entryid_to_append["RateLaw"] = rate_law
                        dict_entryid_to_append["SubstitutedRateLaw"] = rate_law
                    else:
                        dict_entryid_to_append["RateLaw"] = "NaN"
                        dict_entryid_to_append["SubstitutedRateLaw"] = "NaN"

                    if entry_id_flag:

                        fields_to_copy = ["Buffer", "Product", "PubMedID", "Publication", "pH", "Temperature", "Enzyme Variant", "UniProtKB_AC", "Organism", "KineticMechanismType", "SabioReactionID"]
                        for field in fields_to_copy:  
                            dict_entryid_to_append[field] = head_of_df[field]
                        dict_reactions_to_append[entryid] = dict_entryid_to_append
                        dict_entryid_to_append["Substrates"] = head_of_df["Substrate"].split(";")
                        out_rate_law = rate_law
                        if not rate_law in bad_rate_laws:                    
                            substrates = head_of_df["Substrate"].split(";")

                            stripped_string = re.sub('[0-9]', '', rate_law)

                            variables = re.split("\^|\*|\+|\-|\/|\(|\)| ", stripped_string)
                            variables = ' '.join(variables).split()

                            start_value_permutations = ["start value", "start val."]
                            substrates_key = {}
                            for var in variables:
                                if var in parameter_info:
                                    for permutation in start_value_permutations:
                                        try:
                                            if var == "A" or var == "B":
                                                substrates_key[var] = parameter_info[var]["species"]
                                            else:
                                                value = parameter_info[var][permutation]
                                                if value != "-" and value != "" and value != " ":
                                                    out_rate_law = out_rate_law.replace(var, parameter_info[var][permutation])
                                        except:
                                            pass

                            dict_entryid_to_append["RateLawSubstrates"] = substrates_key
                            dict_entryid_to_append["SubstitutedRateLaw"] = out_rate_law

                dict_to_append[reaction] = dict_reactions_to_append

            enzyme_dict[enzyme] = dict_to_append

        with open(scraped_model_json_file_path, 'w') as f:
            json.dump(enzyme_dict, f, indent=4)
        
        self.step_number = 5
        self.progress_update(self.step_number)
        
    def progress_update(self, step):
        if not re.search('[0-5]', str(step)):
            print(f'--> ERROR: The {step} step is not acceptable.')
        f = open(self.paths['progress_file_path'], "w")
        f.write(str(step))
        f.close()

    def main(self,):
        self.start()

        while True:
            if self.step_number == 1:
                self.scrape_bigg_xls()
            elif self.step_number == 2:
                self.glob_xls_files()
            elif self.step_number == 3:
                self.scrape_entryids()
            elif self.step_number == 4:
                self.combine_data()
            elif self.step_number == 5:
                print("Execution complete. Scraper finished.")
                break

Specify the BIGG Model JSON file path: ./BiGG_models/BiGG model of S. aureus.json
C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\dFBApy\scraping\SABIO\BiGG_models


C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: DeprecationWarning: use options instead of chrome_options


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
driver.get("http://sabiork.h-its.org/newSearch/index")
